# Environment Initialization

In [1]:
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from tensorflow import keras

import os
import shutil
if not os.path.exists("./report/figures/"):
  os.makedirs("./report/figures/")
if not os.path.exists("./models/"):
  os.makedirs("./models/")

2023-11-16 16:47:37.847468: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 16:47:37.872429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 16:47:37.872452: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 16:47:37.872457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 16:47:37.876680: I tensorflow/core/platform/cpu_feature_g

# Data Import/Pre-Processing/Visualization

In [2]:
dtype = {
  "Datetime": "string",
  "solar_mw": np.float32,
  "wind-direction": np.float32, 
  "wind-speed": np.float32,
  "humidity": np.float32,
  "average-wind-speed-(period)": np.float32,
  "average-pressure-(period)": np.float32,
  "temperature": np.float32
}
solar_energy_df = pd.read_csv("./solarenergy.csv",
                              delimiter=",",
                              dtype=dtype)

solar_energy_df["Datetime"] = pd.to_datetime(solar_energy_df["Datetime"],
                                             format="%d/%m/%Y %H:%M")
solar_energy_df = solar_energy_df.set_index("Datetime").sort_index()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Raw)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_raw.svg")
plt.close("all")

solar_energy_df = solar_energy_df.dropna()
solar_energy_df = solar_energy_df.resample("1H").interpolate("linear")

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Clean)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_clean.svg")
plt.close("all")

solar_energy_df = \
  (solar_energy_df - solar_energy_df.mean()) / solar_energy_df.std()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Normalized)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_norm.svg")
plt.close("all")

# Baseline Model

In [3]:
training_ratio = 0.7
training_limit = \
  solar_energy_df.index[
      int(training_ratio*solar_energy_df.shape[0])
    ]

best_rmse = np.inf
arimax_model = None

x_train_df = solar_energy_df.iloc[
                :int(training_ratio*solar_energy_df.shape[0]),
                solar_energy_df.columns != "solar_mw"
              ]
y_train_df = solar_energy_df.loc[:training_limit-pd.Timedelta(hours=1),
                                  "solar_mw"]

x_test_df = solar_energy_df.iloc[
              int(training_ratio*solar_energy_df.shape[0]):,1:
            ]
y_test_df = solar_energy_df.loc[training_limit:, "solar_mw"]

for p in [5]:
  for d in [0]:
    for q in [3]:
      try:
        new_model = ARIMA(
          endog=y_train_df,
          exog=x_train_df,
          order=(p,d,q)
        ).fit()
      except:
        continue

      new_prediction = new_model.get_forecast(
          steps=solar_energy_df.index[-1],
          exog=x_test_df).predicted_mean.to_numpy()
      
      new_rmse = np.sqrt(np.mean((new_prediction-y_test_df.to_numpy())**2))
      if new_rmse < best_rmse:
        best_rmse = new_rmse
        arimax_model = new_model

baseline_fit = arimax_model.fittedvalues

baseline_prediction = arimax_model.get_forecast(
    steps=solar_energy_df.index[-1],
    exog=x_test_df
  ).predicted_mean

print(arimax_model.summary())
p,d,q = arimax_model.specification["order"]
print(f"Baseline RMSE: {best_rmse}")
print(f"Baseline MAE: {np.abs(baseline_prediction-y_test_df.to_numpy()).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(baseline_fit, label="Fitted")
plt.plot(baseline_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"ARIMAX({p},{d},{q})")
plt.legend()
plt.savefig("./report/figures/baseline_fit.svg")
plt.close("all")

/home/sergiovaneg/miniforge3/envs/ADAML/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:               solar_mw   No. Observations:                 1544
Model:                 ARIMA(5, 0, 3)   Log Likelihood                1884.828
Date:                Thu, 16 Nov 2023   AIC                          -3737.655
Time:                        16:47:45   BIC                          -3652.181
Sample:                    05-03-2020   HQIC                         -3705.859
                         - 07-06-2020                                         
Covariance Type:                  opg                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.0943      0.123      0.764      0.445      -0.148       0.336
wind-direction                 -0.0004      0.004     -0.098      0.922      -0.

# Recursive Models

## Data Preparation

In [4]:
n_feats = len(x_train_df.columns)

def tensor_memory_reshaper(in_np:np.ndarray, out_np:np.ndarray,
                           memory:int, n_feats_internal:int = n_feats):
  in_np = in_np.reshape((-1,1,n_feats_internal))
  for _ in range(memory):
    next_np = in_np[1:,-1,:].reshape(((-1,1,n_feats_internal)))
    in_np = np.concatenate((in_np[:-1,:,:], next_np), axis=1)
  return (tf.convert_to_tensor(in_np),
          tf.convert_to_tensor(out_np[memory:]))

x_train_np = np.pad(x_train_df.to_numpy(), ((0,0),(0,0)), mode="edge")
y_train_np = np.pad(y_train_df.to_numpy(), ((0,0),), mode="edge")

x_test_np = pd.concat((x_train_df.iloc[-q:], x_test_df)).to_numpy()
y_test_np = pd.concat((y_train_df.iloc[-q:], y_test_df)).to_numpy()
x_test_np = x_test_df.to_numpy()
y_test_np = y_test_df.to_numpy()

x_train_tensor,y_train_tensor = \
  tensor_memory_reshaper(x_train_np, y_train_np, 0)
x_test_tensor,y_test_tensor = \
  tensor_memory_reshaper(x_test_np, y_test_np, 0)

class StateResetCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs=None):
    self.model.reset_states()

2023-11-16 16:47:47.513418: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-16 16:47:47.532203: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-16 16:47:47.532386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Simple RNN

In [5]:
if not os.path.exists("./models/rnn_model.keras"):
  rnn_model = keras.Sequential()
  for _ in range(p-1):
    rnn_model.add(keras.layers.SimpleRNN(q, stateful=True,
                                         return_sequences=True,
                                         batch_input_shape=(1,1,n_feats)))
  rnn_model.add(keras.layers.SimpleRNN(q, stateful=True))
  rnn_model.add(keras.layers.Dense(1, keras.activations.linear))

  rnn_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/rnn_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )
  

  rnn_model.compile(optimizer="Adam", loss="mse")
  rnn_model.fit(x_train_tensor, y_train_tensor,
                shuffle=False, batch_size=1,
                epochs=100, verbose="auto",
                callbacks=[rnn_model_checkpoint, StateResetCallback()])
  
  rnn_model = keras.models.load_model("./models/rnn_model_checkpoint")
  shutil.rmtree("./models/rnn_model_checkpoint")
  rnn_model.save("./models/rnn_model.keras")
else:
  rnn_model = keras.models.load_model("./models/rnn_model.keras")

rnn_model.reset_states()
rnn_fit = rnn_model.predict(x_train_tensor, batch_size=1)
rnn_prediction = rnn_model.predict(x_test_tensor, batch_size=1)

print(f"RNN RMSE: {np.sqrt(np.mean((rnn_prediction - y_test_tensor)**2))}")
print(f"RNN MAE: {np.abs(rnn_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, rnn_fit, label="Fitted")
plt.plot(y_test_df.index, rnn_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"Simple RNN")
plt.legend()
plt.savefig("./report/figures/rnn_fit.svg")
plt.close("all")

No such comm: 2c5aa7123ab848f9bc551e87cfce1bf6


2023-11-16 16:47:47.688464: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/100


2023-11-16 16:47:49.950639: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdbd51321f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-16 16:47:49.950661: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A500 Laptop GPU, Compute Capability 8.6
2023-11-16 16:47:49.953675: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-16 16:47:49.961611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2023-11-16 16:47:50.000777: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1540/1544 [============================>.] - ETA: 0s - loss: 0.9870INFO:tensorflow:Assets written to: ./models/rnn_model_checkpoint/assets


...

1544/1544 [==============================] - 16s 10ms/step - loss: 0.8188
Epoch 50/100
1540/1544 [============================>.] - ETA: 0s - loss: 0.8143INFO:tensorflow:Assets written to: ./models/rnn_model_checkpoint/assets


INFO:tensorflow:Assets written to: ./models/rnn_model_checkpoint/assets


1544/1544 [==============================] - 16s 10ms/step - loss: 0.8148
Epoch 51/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8190
Epoch 52/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8223
Epoch 53/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8378
Epoch 54/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8424
Epoch 55/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8512
Epoch 56/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8376
Epoch 57/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8519
Epoch 58/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8510
Epoch 59/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8408
Epoch 60/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8436
Epoch 61/100
1544/1544 [==============================] - 14s 9ms/

INFO:tensorflow:Assets written to: ./models/rnn_model_checkpoint/assets


1544/1544 [==============================] - 16s 10ms/step - loss: 0.8053
Epoch 64/100
1544/1544 [==============================] - 13s 9ms/step - loss: 0.8348
Epoch 65/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8268
Epoch 66/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8372
Epoch 67/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8512
Epoch 68/100
1544/1544 [==============================] - 12s 8ms/step - loss: 0.8429
Epoch 69/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8376
Epoch 70/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8395
Epoch 71/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8510
Epoch 72/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8543
Epoch 73/100
1544/1544 [==============================] - 14s 9ms/step - loss: 0.8535
Epoch 74/100
1544/1544 [==============================] - 14s 9ms/

## LSTM

In [7]:
if not os.path.exists("./models/lstm_model.keras"):
  lstm_model = keras.Sequential()
  for _ in range(p-1):
    lstm_model.add(keras.layers.LSTM(q, stateful=True,
                                         return_sequences=True,
                                         batch_input_shape=(1,1,n_feats)))
  lstm_model.add(keras.layers.LSTM(q, stateful=True))
  lstm_model.add(keras.layers.Dense(1, keras.activations.linear))

  lstm_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/lstm_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )

  lstm_model.compile(optimizer="Adam", loss="mse")
  lstm_model.fit(x_train_tensor, y_train_tensor,
                shuffle=False, batch_size=1,
                epochs=100, verbose="auto",
                callbacks=[lstm_model_checkpoint, StateResetCallback()])
  
  lstm_model = keras.models.load_model("./models/lstm_model_checkpoint")
  shutil.rmtree("./models/lstm_model_checkpoint")
  lstm_model.save("./models/lstm_model.keras")
else:
  lstm_model = keras.models.load_model("./models/lstm_model.keras")

lstm_model.reset_states()
lstm_fit = lstm_model.predict(x_train_tensor, batch_size=1)
lstm_prediction = lstm_model.predict(x_test_tensor, batch_size=1)

print(f"LSTM RMSE: {np.sqrt(np.mean((rnn_prediction - y_test_tensor)**2))}")
print(f"LSTM MAE: {np.abs(rnn_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, lstm_fit, label="Fitted")
plt.plot(y_test_df.index, lstm_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"LSTM")
plt.legend()
plt.savefig("./report/figures/lstm_fit.svg")
plt.close("all")

Epoch 1/100
1543/1544 [============================>.] - ETA: 0s - loss: 0.8606INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


1544/1544 [==============================] - 34s 20ms/step - loss: 0.8602
Epoch 2/100
1544/1544 [==============================] - 26s 17ms/step - loss: 0.9260
Epoch 3/100
1544/1544 [==============================] - 25s 16ms/step - loss: 0.8720
Epoch 4/100
1544/1544 [==============================] - 24s 15ms/step - loss: 0.8825
Epoch 5/100
1544/1544 [==============================] - 21s 14ms/step - loss: 0.8995
Epoch 6/100
1544/1544 [==============================] - 20s 13ms/step - loss: 0.9007
Epoch 7/100
1544/1544 [==============================] - 21s 13ms/step - loss: 0.8661
Epoch 8/100
1544/1544 [==============================] - 20s 13ms/step - loss: 0.8830
Epoch 9/100
1544/1544 [==============================] - 25s 16ms/step - loss: 0.8761
Epoch 10/100
1544/1544 [==============================] - ETA: 0s - loss: 0.8508INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


1544/1544 [==============================] - 24s 15ms/step - loss: 0.8508
Epoch 11/100
1544/1544 [==============================] - 20s 13ms/step - loss: 0.8515
Epoch 12/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.8644
Epoch 13/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.8570
Epoch 14/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.8600
Epoch 15/100
1544/1544 [==============================] - ETA: 0s - loss: 0.8450INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


...

1544/1544 [==============================] - 23s 15ms/step - loss: 0.7455
Epoch 66/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7508
Epoch 67/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7649
Epoch 68/100
1543/1544 [============================>.] - ETA: 0s - loss: 0.7415INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


1544/1544 [==============================] - 23s 15ms/step - loss: 0.7410
Epoch 69/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7662
Epoch 70/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7670
Epoch 71/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7599
Epoch 72/100
1543/1544 [============================>.] - ETA: 0s - loss: 0.7363INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


INFO:tensorflow:Assets written to: ./models/lstm_model_checkpoint/assets


1544/1544 [==============================] - 23s 15ms/step - loss: 0.7359
Epoch 73/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7561
Epoch 74/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7571
Epoch 75/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7407
Epoch 76/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7380
Epoch 77/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7426
Epoch 78/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7392
Epoch 79/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7479
Epoch 80/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7484
Epoch 81/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7836
Epoch 82/100
1544/1544 [==============================] - 17s 11ms/step - loss: 0.7654
Epoch 83/100
1544/1544 [==============================] 